# Problema de la mochila
## Definición de la función

In [36]:
import numpy as np

MOCHILA = 1
K_PENALIZACION = 20

def evaluar(peso: np.ndarray, valor: np.ndarray):
    exceso = MOCHILA - peso
    evaluacion = valor.copy()
    evaluacion[exceso < 0] = evaluacion[exceso < 0] - np.abs(evaluacion[exceso < 0] * exceso[exceso < 0]) - K_PENALIZACION
    return evaluacion

## Definición de cromosoma
Se compone de una lista que representa los objetos que lleva

In [37]:
NUM_OBJETOS = 20
VALORES = np.random.randint(low=1, high=101, size=NUM_OBJETOS)
PESOS = np.random.random(size=NUM_OBJETOS)      # Puede regresar 0
PESOS[PESOS == 0] = 0.1

class Cromosoma:
    """
    Se conforma por una lista que sirve como índice lógico para PESOS y VALORES
    """
    def __init__(self, objetos = None):
        if objetos is None:
            self.list = np.random.choice([0, 1], size=NUM_OBJETOS)
        else:
            self.list = objetos

    def __str__(self):
        return str(self.list)


    def get_peso(self):
        return np.sum(PESOS[self.list == 1])

    def get_valor(self):
        return np.sum(VALORES[self.list == 1])

    @staticmethod
    def crossover(c1, c2):
        mitad = int(NUM_OBJETOS / 2)
        l1 = np.append(c1.list[0:mitad], c2.list[mitad:], axis=None)
        l2 = np.append(c2.list[0:mitad], c1.list[mitad:], axis=None)
        return [Cromosoma(l1), Cromosoma(l2)]

    @staticmethod
    def mutar(c):
        index = np.random.randint(low=0, high=NUM_OBJETOS)
        c.list[index] = 1 - c.list[index]


## Definición de presión selectiva

In [38]:
K_POBLACION = 8
K_BASE = 2
K_PROBABILIDAD_MUTACION = 0.5

def presion_selectiva(poblacion: list[Cromosoma]) -> list[Cromosoma]:
    # Evaluación y búsqueda del mejor
    c_pesos = np.array([c.get_peso() for c in poblacion])
    c_valores = np.array([c.get_valor() for c in poblacion])
    evaluacion = evaluar(c_pesos, c_valores)

    best = evaluacion.argmax()
    print("Best so far:")
    print(f"Combination: {poblacion[best]}")
    print(f"Weight: {c_pesos[best]}")
    print(f"Value: {c_valores[best]}")
    print(f"Evaluation: {evaluacion[best]}")

    # Cálculo de probabilidades
    indice_ordenado = (-evaluacion).argsort()
    ruleta = []
    potencia = K_POBLACION

    for i in indice_ordenado:
        probabilidad = K_BASE ** potencia
        ruleta.extend([i] * probabilidad)
        potencia -= 1

    # Nueva generación
    nueva = list[Cromosoma]()
    nueva.append(poblacion[indice_ordenado[0]])
    nueva.append(poblacion[indice_ordenado[1]])

    for i in range(1, int(K_POBLACION/2)):
        c1 = poblacion[np.random.choice(ruleta)]
        c2 = poblacion[np.random.choice(ruleta)]
        hijos = Cromosoma.crossover(c1, c2)

        for hijo in hijos:
            if np.random.choice([True, False], p=[K_PROBABILIDAD_MUTACION, 1-K_PROBABILIDAD_MUTACION]):
                Cromosoma.mutar(hijo)

        nueva.extend(hijos)

    return nueva

## Ciclo de vida

In [39]:
poblacion = list[Cromosoma]()
nueva_poblacion = list[Cromosoma]()
generacion = 0

In [53]:
if len(nueva_poblacion) == 0:
    poblacion = [Cromosoma() for _ in range(0, K_POBLACION)]
else:
    poblacion = nueva_poblacion

print('Generation', generacion)
nueva_poblacion = presion_selectiva(poblacion)
generacion += 1

Generation 13
Best so far:
Combination: [0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0]
Weight: 0.976136194500251
Value: 314
Evaluation: 314
